# Building RAG from Scratch
https://docs.llamaindex.ai/en/stable/examples/low_level/oss_ingestion_retrieval/

## Model Define

In [24]:
SENTENCE_TRANSFORMER = 'sentence-transformers/all-MiniLM-L6-v2'
LLM = 'llama3-70b-8192'
LLM_API_KEY = 'your-key'
DOCUMENT = '../data/llama2.pdf'

## Load Model

In [25]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq

embed_model = HuggingFaceEmbedding(model_name=SENTENCE_TRANSFORMER)
llm = Groq(model=LLM, api_key=LLM_API_KEY)

/home/telan3/miniconda3/envs/rag_tutorials/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Initialize Postgres

In [28]:
import psycopg2

db_name = "vector_db"
host = "192.168.112.12"
password = "watchall"
port = "5432"
user = "watchtek"
conn = psycopg2.connect(
    dbname="postgres",
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS {db_name}")
    c.execute(f"CREATE DATABASE {db_name}")

In [29]:
from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="llama2_paper",
    embed_dim=384,  # openai embedding dimension
)

## Build an Ingestion Pipeline from Scratch

### 1. Load Data

In [31]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader
loader = PyMuPDFReader()
documents = loader.load(file_path=DOCUMENT)

### 2. Use a Text Splitter to Split Documents

In [32]:
from llama_index.core.node_parser import SentenceSplitter
text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [33]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

### 3. Manually Construct Nodes from Text Chunks

In [34]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

### 4. Generate Embeddings for each Node

In [35]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

### 5. Load Nodes into a Vector Store

In [36]:
vector_store.add(nodes)

['0de85b78-ce62-4bb0-87cb-80130422faa4',
 'b479f4e4-f2c7-4129-83b9-8d77f75a75f3',
 '46efff11-da25-4dae-96db-10422e582e24',
 'aef71ffa-ba5f-4505-92a2-6242b72b2807',
 'dc0cf73c-3b4a-4acc-8bfe-739cd8534f58',
 'a1b631b0-a270-497f-af48-379a10aee84a',
 'b5ab3f9f-ff87-4bd7-8b07-1437fbce0321',
 '4fe7c645-a07a-4bd7-8080-f076c450b864',
 '19133a25-9652-4d46-9cda-8c43090056f1',
 '92304442-bc40-4ba2-9bcb-5464bed4f06c',
 '27affe01-0d27-4e25-8f2e-1f2d82df1218',
 'e216f022-5084-4c75-b282-fb02cbc64fba',
 '173140b8-be32-4a85-9c0c-0a18a9edae16',
 'fa860ded-dd88-4d10-ad9c-299173a70514',
 '26285045-d088-4778-8375-d84f97da2677',
 'cad6fbfd-ed92-4ec5-aee2-b67306502e7b',
 '929b5bba-121a-4d0c-b585-623725e1e306',
 '45f0523e-b5b2-47a7-ab69-0d71ed8e0274',
 '84653add-7234-40cd-a470-e6faf351e370',
 '8cc1f510-7c50-49f9-b99b-9627129d7d04',
 '2449b4c5-e16d-48b8-a4f0-ba1744e43373',
 '47197e59-9066-40ce-b04c-1d93daf25b31',
 '819ec27d-fdb8-4c17-b00d-9957c077090f',
 '051e8d15-8f99-4b95-a926-79f598827190',
 '1cbfbbbe-0f10-

## Build Retrieval Pipeline from Scratch

In [37]:
query_str = "Can you tell me about the key concepts for safety finetuning"

### 1. Generate a Query Embedding

In [38]:
query_embedding = embed_model.get_query_embedding(query_str)

### 2. Query the Vector Database

In [39]:
# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [40]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())

4
Safety
In this section, we dive deeper into the important topic of safety measurements and mitigations. We first
discuss our safety investigations into pretraining data and pretrained models (Section 4.1). Next, we describe
the process of our safety alignment (Section 4.2), explaining how we collected safety-related annotations and
utilized SFT and RLHF, and present experimental results. Then, we discuss the red teaming we performed to
further understand and improve model safety (Section 4.3). Finally, we present quantitative safety evaluations
of Llama 2-Chat (Section 4.4). We also share a model card in the Appendix, in Table 52.
4.1
Safety in Pretraining
It is important to understand what is in the pretraining data both to increase transparency and to shed
light on root causes of potential downstream issues, such as potential biases. This can inform what, if any,
downstream mitigations to consider, and help guide appropriate model use. In this section, we analyze the
pretraining da

### 3. Parse Result into a Set of Nodes

In [41]:
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

### 4. Put into a Retriever

In [42]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [43]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

## Plug this into our RetrieverQueryEngine to synthesize a response

In [44]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [45]:
query_str = "How does Llama 2 perform compared to other open-source models?"

response = query_engine.query(query_str)
print(str(response))

Llama 2 70B outperforms all open-source models.


In [46]:
print(response.source_nodes[0].get_content())

Additionally, Llama 2 70B model outperforms all open-source models.
In addition to open-source models, we also compare Llama 2 70B results to closed-source models. As shown
in Table 4, Llama 2 70B is close to GPT-3.5 (OpenAI, 2023) on MMLU and GSM8K, but there is a significant
gap on coding benchmarks. Llama 2 70B results are on par or better than PaLM (540B) (Chowdhery et al.,
2022) on almost all benchmarks. There is still a large gap in performance between Llama 2 70B and GPT-4
and PaLM-2-L.
We also analysed the potential data contamination and share the details in Section A.6.
Benchmark (shots)
GPT-3.5
GPT-4
PaLM
PaLM-2-L
Llama 2
MMLU (5-shot)
70.0
86.4
69.3
78.3
68.9
TriviaQA (1-shot)
–
–
81.4
86.1
85.0
Natural Questions (1-shot)
–
–
29.3
37.5
33.0
GSM8K (8-shot)
57.1
92.0
56.5
80.7
56.8
HumanEval (0-shot)
48.1
67.0
26.2
–
29.9
BIG-Bench Hard (3-shot)
–
–
52.3
65.7
51.2
Table 4: Comparison to closed-source models on academic benchmarks. Results for GPT-3.5 and GPT-4
are from OpenAI